In [1]:
import pandas as pd  
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from scipy import stats
from ast import literal_eval # 문자열 모형의 딕트를 스근하게 딕트로 바꾸어 준다. 
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from surprise import Reader, Dataset, SVD, evaluate

In [31]:
original_data = pd.read_csv('./eyeShadow.csv')
type = {'건성': 0, '지성': 1, '중성': 2, '복합성': 3, '민감성': 4}
original_data['type'] = original_data['type'].map(type)

In [48]:
id_purify_data = original_data.copy()
id_purify_data.head(3)

,userId,name,popId,rate,review,type
0,0,매트 아이 컬러,0,4.57,진저 진짜 짱 추천합니다! 코섀디에도 완전 자연스럽고 그냥 단독으로도 진짜 이뻐요?...,0
1,1,매트 아이 컬러,0,4.57,진저....평생이걸로쓸거에요,0
2,2,매트 아이 컬러,0,4.57,완전 매트하고 다 제가 좋아하는 색이여서 너무 좋아요!,0


In [49]:
member_per_type = {0: 864, 1: 849, 2: 775, 3: 855, 4: 835}
for i in range(5):
    np.random.seed(42)
    id_purify_data.loc[id_purify_data["type"] == i, "userId"] = [np.random.randint(i*200, 200*(i+1))
                                                             for j in range(0, member_per_type[i])]


In [50]:
id_purify_data.head()

,userId,name,popId,rate,review,type
0,53,매트 아이 컬러,0,4.57,진저 진짜 짱 추천합니다! 코섀디에도 완전 자연스럽고 그냥 단독으로도 진짜 이뻐요?...,0
1,190,매트 아이 컬러,0,4.57,진저....평생이걸로쓸거에요,0
2,86,매트 아이 컬러,0,4.57,완전 매트하고 다 제가 좋아하는 색이여서 너무 좋아요!,0
3,173,매트 아이 컬러,0,4.57,좋은 거 하나 사서 힛팬 하는 걸 목표로 프레스코 구매함. 프레스코 저렴이 많이 가...,0
4,111,매트 아이 컬러,0,4.57,한창 섀도우에 빠져 있을때 뭐 살지 고민하다가 평이 좋아서 진저 구매해서 사용해 봤...,0


In [51]:
id_purify_data = id_purify_data.sort_values('userId')

In [52]:
id_purify_data.head(3)

,userId,name,popId,rate,review,type
315,0,네이키드 히트,7,4.40,외국 브랜드 섀도우라서 로드샵의 발색을 생각했다면 좀 당황하실 수도 있어요. 색은 ...,0
3019,0,롱웨어 크림 섀도우 스틱,68,4.31,발림성 굳굳 오래 놔둬도 잘 안굳어서 좋아요\r\n애교랑 언더쪽에 음영감 넣고싶을때...,0
1911,0,플레이 컬러 아이즈 체리 블라썸,43,4.30,컬러 완전취저\r\n4계절 계속 쓰기 좋은 컬러로 구성되 있어서 짱이에요ㅎㅎ\r\n...,0


In [104]:
def reviewData(type=0):
    review_data = ['' for i in range(96)]
    review_type_popid_data = pd.DataFrame(id_purify_data[["popId", "type", "review"]])
    review_type_popid_data = review_type_popid_data.reset_index(drop=True)
    review_type_popid_data = review_type_popid_data.loc[review_type_popid_data["type"]==type,:]
    
    for index, row in review_type_popid_data.iterrows():
        review_data[row["popId"]] += row["review"]
   
    return review_data

In [108]:
review_data = reviewData(0)


'색상 #앤티크로즈 #토스트\r\n앤티크로즈 색깔 진짜 이뻐요 원래 토스트만 살 생각이였는데 이 색깔 보지 하고 손등에 발라봤는데 너무 이뻐서 큰맘먹고 샀어요 근데 그 이후로 섀도우 단 1개도 안사고 계속 앤티크로즈랑 토스트만 발라요!! 진짜 면세점 찬스로 샀는데 면세점이 더 저렴하니깐 면세점 찬스를 이용하시길 바래요!! 그리고 앤티크로즈를 베이스로 깔고 토스트를 끝라인으로 해서 빼주면 진짜 짱!! 그리고 다른제품으로 애굣살에 펄을 발라주면 진짜 제 인생메이크업이예요!! 완전 추천날씨가 쌀쌀해지니 토스트. 앤티크로즈 바르고 싶어져서\r\n섞발해 쓰고 루비 연하게 발라주면 너무 예쁩니당가을웜톤(트루가을)입니다\r\n#토스트 사용하는데 만족스러워요\r\n저는 로라메르시에 진저를 베이스로 깔고\r\n얘를 쌍커풀라인으로 바르는데 예쁘네요\r\n근데 굳이 재구매를 할 이유는 없을것같아요\r\n\r\n제품의 질은 우수하지만 다른것들로도 대체할수있다고 생각합니다앤티크로즈, 헤더 사용했습니다\r\n색상은 참 예뻐요\r\n로라에 비해 발색이 잘 안되긴 함 ㅠ 발림성도 그저그렇고\r\n그래도 색이 독보적이라 사용할수밖에...\r\n데일리로 매일 쓰는 조합 ㅠㅠ샴페인 쿼츠 영원히 함께하고파...?? 웜쿨톤 모두 쓸 수 있는 섀도우!! 눈동자 중앙에 톡톡 눈밑에 톡톡 발라주면 이세상 고급짐이 아니다... 하... 진짜 비싼 느낌이 난댜... 부내가 난댜... 바비브라운꺼 산 뒤로는 로드샵 섀도우에 눈이 안감 ㅠㅠㅠ 확실히 발색력 지속력이 다른듯 ㅠㅠㅠ 재구매의사 200%임진짜 믿고 쓰는 엔티크로즈ㅠㅠㅠㅠㅠㅠ그냥 엔티크 로즈 하나만 발라도 완성입니다 여러분 꼭 사세요ㅠㅠㅠ양도 많아서 계속 써도 힛팬 보이기 어려운 제품인 것 같아요 가루 날림도 없어서 진짜 섀도우 중 갑입니다ㅠㅠㅠ우선용량부터맘에들어용이번에처음으로바비브라운섀도우샀는데발림성도너무조코가루날림이없어서조아요????케이스가 다했음. 용량이 너무 많음ㅠ.ㅠ 그치만 챡챡 보관하기 좋고 색 잘 보여서 좋음. 아이섀도우는 앵간하면 다 